In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/avjobathonmay2021/train_s3TEQDk.csv
/kaggle/input/avjobathonmay2021/sample_submission_eyYijxG.csv
/kaggle/input/avjobathonmay2021/test_mSzZ8RL.csv


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [5]:
train = pd.read_csv('/kaggle/input/avjobathonmay2021/train_s3TEQDk.csv')
test = pd.read_csv('/kaggle/input/avjobathonmay2021/test_mSzZ8RL.csv')
sample = pd.read_csv('/kaggle/input/avjobathonmay2021/sample_submission_eyYijxG.csv')

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 245725 entries, 0 to 245724
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ID                   245725 non-null  object
 1   Gender               245725 non-null  object
 2   Age                  245725 non-null  int64 
 3   Region_Code          245725 non-null  object
 4   Occupation           245725 non-null  object
 5   Channel_Code         245725 non-null  object
 6   Vintage              245725 non-null  int64 
 7   Credit_Product       216400 non-null  object
 8   Avg_Account_Balance  245725 non-null  int64 
 9   Is_Active            245725 non-null  object
 10  Is_Lead              245725 non-null  int64 
dtypes: int64(4), object(7)
memory usage: 20.6+ MB


In [7]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105312 entries, 0 to 105311
Data columns (total 10 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ID                   105312 non-null  object
 1   Gender               105312 non-null  object
 2   Age                  105312 non-null  int64 
 3   Region_Code          105312 non-null  object
 4   Occupation           105312 non-null  object
 5   Channel_Code         105312 non-null  object
 6   Vintage              105312 non-null  int64 
 7   Credit_Product       92790 non-null   object
 8   Avg_Account_Balance  105312 non-null  int64 
 9   Is_Active            105312 non-null  object
dtypes: int64(3), object(7)
memory usage: 8.0+ MB


In [8]:
sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105312 entries, 0 to 105311
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   ID       105312 non-null  object
 1   Is_Lead  105312 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.6+ MB


In [9]:
y = train['Is_Lead']

In [10]:
y.value_counts()

0    187437
1     58288
Name: Is_Lead, dtype: int64

In [11]:
train.drop(['ID','Is_Lead'], axis=1, inplace=True)
test.drop(['ID'], axis=1, inplace=True)

In [12]:
train['TRV'] = train['Vintage'] * train['Avg_Account_Balance']
test['TRV'] = test['Vintage'] * test['Avg_Account_Balance']

In [13]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 245725 entries, 0 to 245724
Data columns (total 10 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   Gender               245725 non-null  object
 1   Age                  245725 non-null  int64 
 2   Region_Code          245725 non-null  object
 3   Occupation           245725 non-null  object
 4   Channel_Code         245725 non-null  object
 5   Vintage              245725 non-null  int64 
 6   Credit_Product       216400 non-null  object
 7   Avg_Account_Balance  245725 non-null  int64 
 8   Is_Active            245725 non-null  object
 9   TRV                  245725 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 18.7+ MB


In [14]:
import pandas_profiling

In [15]:
pandas_profiling.ProfileReport(train)

Summarize dataset:   0%|          | 0/24 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from category_encoders import TargetEncoder
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

numeric_features = [1, 5, 7,9]
numeric_transformer = Pipeline(steps=[('imputer', IterativeImputer(random_state=1)),
    ('scaler', StandardScaler())])

categorical_features = [0, 2, 3, 4, 6, 8]
# categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='constant', fill_value='No_Idea')),
#     ('onehot', OneHotEncoder(handle_unknown='ignore'))])
categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='constant', fill_value='No_Idea')),
   ('Target encoding', TargetEncoder())])
preprocessor = ColumnTransformer(transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
        ])

In [ ]:
#split training set for training and testing
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(train, y, random_state=1, test_size=0.3, shuffle=True, stratify=y)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

In [ ]:
from sklearn.pipeline import make_pipeline, Pipeline
from catboost import CatBoostClassifier

model = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', CatBoostClassifier(class_weights=[3.21570477628,1], iterations=500, learning_rate=0.1, depth=5, random_state=2021))])
model.fit(X_train, y_train)
print(model.score(X_train, y_train))

In [ ]:
y_pred = model.predict(X_val)
print(model.score(X_val, y_val))

0.8572397514853902

In [ ]:
model = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', CatBoostClassifier(class_weights=[0.75, 0.25], iterations=500, learning_rate=0.1, depth=5, random_state=2021))])
model.fit(train, y)

In [ ]:
pred = model.predict_proba(test)[:,1]

In [ ]:
sample['Is_Lead'] = pred

In [ ]:
sample.describe(include='all').T

In [ ]:
sample.to_csv('submission_six.csv', index=False)

In [ ]:
sample.head()